# 02wk-1: Overview (3)

최규빈  
2022-09-13

> fastai를 이용한 분석 steps, 추천시스템 실습, 텍스트분석 실습

# 강의영상

> <https://youtube.com/playlist?list=PLQqh36zP38-zJbe5FztJPSqicA2ZLwDgh>

# imports

In [2]:
from fastai.collab import * ## 추천시스템
from fastai.text.all import * ## 텍스트분석 

In [3]:
import pandas as pd

# fastai를 이용한 분석 steps

`-` 비교

|       |        이미지분석(CNN)        |    추천시스템     |        텍스트분석        |        GAN        |
|:-------------:|:-------------:|:-------------:|:-------------:|:-------------:|
| 1단계 |       ImageDataLoaders        | CollabDataLoaders |     TextDataLoaders      | DataBlock -\> dls |
| 2단계 |         cnn_learner()         | collab_learner()  | language_model_learner() | GANLearner.wgan() |
| 3단계 |       lrnr.fine_tune(1)       |    lrnr.fit()     |        lrnr.fit()        |    lrnr.fit()     |
| 4단계 | lrnr.predict(), lrnr.model(X) |   lrnr.model(X)   |      lrnr.predict()      |                   |

# 추천시스템 실습

## 1단계

In [7]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/main/posts/I.%20Overview/2022-09-08-rcmd_view.csv')
df_view

-   row0 - row49 에 해당하는 유저는 커피를 선호
-   row50 - row99 에 해당하는 유저는 홍차를 선호

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/main/posts/I.%20Overview/2022-09-08-rcmd_anal.csv')
df

-   컴퓨터는 이러한 형태를 더 분석하기 좋아한다.

In [115]:
df.item.unique(),df.user.unique()
# 유저는 1~100 으로 아이템은 1~20으로 번호가 매겨져 있음 

In [116]:
dls=CollabDataLoaders.from_df(df)

In [127]:
dls.show_batch()

In [87]:
X,y= dls.one_batch()

In [88]:
X[0],y[0]

-   64번 유저가 15번 아이템을 먹었을때 평점을 4.1146 주었음

## 2단계

In [89]:
lrnr = collab_learner(dls,y_range=(0,5)) # y_range는 평점의 범위

## 3단계

In [92]:
lrnr.fit(10) # 총 30번 정도 해야 적합이 잘된다. 

## 4단계

`-` 하나의 배치 전체를 예측

In [93]:
yhat=lrnr.model(X.to("cuda:0"))
yhat

-   lrnr.model()은 GPU메모리에 존재하고 X는 일반메모리에 존재하므로 X를
    GPU메모리로 옮겨주어야 함
-   X.to(“cuda:0”)을 통하여 X를 GPU메모리로 옮기는 작업을 수행할 수
    있다.

`-` 하나의 유저가 하나의 아이템을 선택했다고 가정하고 예측 (주어진
자료중에서 예측)

In [94]:
X.shape

In [27]:
X[0:1]

-   18번 유저가 5번 아이템(커피)를 먹는다면?

In [26]:
lrnr.model(X[0:1].to("cuda:0"))

-   평점은 4.1128정도 될것

`-` 하나의 유저가 하나의 아이템을 선택했다고 가정하고 예측 (주어지지
않은 자료중에서 예측)

In [29]:
X[0:1]

In [36]:
Xnew = torch.tensor([[1,  2]])

In [37]:
lrnr.model(Xnew.to("cuda:0"))

# 텍스트분석 실습

## 1단계

In [39]:
df = pd.DataFrame({'text':['h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??']*20000})
df

In [44]:
dls = TextDataLoaders.from_df(df,text_col='text',is_lm=True) 

In [45]:
dls.show_batch()

-   is_lm: text의 생성에 관심이 있다면 True로 설정할 것

## 2단계

In [58]:
lrnr = language_model_learner(dls, AWD_LSTM)

## 3단계

In [63]:
lrnr.fit(1)

## 4단계

In [68]:
lrnr.predict('h e',n_words=30)